## Semantic model example
This notebook creates RDF with relevant ontologies found through EBI-OLS: https://www.ebi.ac.uk/ols/index. The semantic model is a reflection of the personal interpretation of Andra Waagmeester who is no expert on the data. The generated linked data should be treated as such, exemplary data of a possible semantic model. This model and the (arbitrary) choices for the selected ontological terms need to be reviewed or redesigned by a field expert. 

In [92]:
from wikidataintegrator import wdi_core, wdi_helpers, wdi_login
import pandas as pd
from rdflib import Graph, URIRef, Literal, BNode, Namespace
from rdflib.namespace import FOAF, RDF, RDFS, DCTERMS, DC, XSD, SKOS

In [103]:
schema = Namespace("http://schema.org/")
patientData = Namespace("http://leap.demonstrator.org/patient/")
tcga = Namespace("https://wiki.nci.nih.gov/display/TCGA/")
wikidata = Namespace("http://www.wikidata.org/entity/")
sio = Namespace("http://semanticscience.org/resource/")
obo = Namespace("http://purl.obolibrary.org/obo/")
snomed = Namespace("http://snomed.info/id/")

In [38]:
df = pd.read_csv("TCGA/curated_nationwidechildrens.org_clinical_patient_brca.txt", sep="\t")
df

,bcr_patient_uuid,bcr_patient_barcode,form_completion_date,prospective_collection,retrospective_collection,birth_days_to,gender,menopause_status,race,ethnicity,history_other_malignancy,history_neoadjuvant_treatment,tumor_status,vital_status,last_contact_days_to,death_days_to,radiation_treatment_adjuvant,pharmaceutical_tx_adjuvant,histologic_diagnosis_other,initial_pathologic_dx_year,age_at_diagnosis,method_initial_path_dx,method_initial_path_dx_other,surgical_procedure_first,first_surgical_procedure_other,margin_status,surgery_for_positive_margins,surgery_for_positive_margins_other,margin_status_reexcision,axillary_staging_method,axillary_staging_method_other,micromet_detection_by_ihc,lymph_nodes_examined,lymph_nodes_examined_count,lymph_nodes_examined_he_count,lymph_nodes_examined_ihc_count,ajcc_staging_edition,ajcc_tumor_pathologic_pt,ajcc_nodes_pathologic_pn,ajcc_metastasis_pathologic_pm,...,nte_er_positivity_define_method,nte_pr_status_by_ihc,nte_pr_status_ihc__positive,nte_pr_ihc_intensity_score,nte_pr_positivity_other_scale,nte_pr_positivity_define_method,nte_her2_status,nte_her2_status_ihc__positive,nte_her2_positivity_ihc_score,nte_her2_positivity_other_scale,nte_her2_positivity_method,nte_her2_fish_status,nte_her2_signal_number,nte_cent_17_signal_number,her2_cent17_counted_cells_count,nte_cent_17_her2_ratio,nte_cent17_her2_other_scale,nte_her2_fish_define_method,anatomic_neoplasm_subdivision,clinical_M,clinical_N,clinical_T,clinical_stage,days_to_initial_pathologic_diagnosis,days_to_patient_progression_free,days_to_tumor_progression,disease_code,extranodal_involvement,histological_type,icd_10,icd_o_3_histology,icd_o_3_site,informed_consent_verified,metastatic_tumor_indicator,patient_id,project_code,site_of_primary_tumor_other,stage_other,tissue_source_site,tumor_tissue_site
0,6E7D5EC6-A469-467C-B748-237353C23416,TCGA-3C-AAAU,2014-1-13,NO,YES,-20211,FEMALE,Pre (<6 months since LMP AND no prior bilatera...,WHITE,NOT HISPANIC OR LATINO,No,No,WITH TUMOR,Alive,3767,[Not Applicable],NO,YES,[Not Applicable],2004,55,[Not Available],[Not Applicable],Modified Radical Mastectomy,[Not Available],Negative,[Not Available],[Not Applicable],[Not Available],Sentinel lymph node biopsy plus axillary disse...,[Not Available],YES,YES,13,4,[Not Available],6th,TX,NX,MX,...,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],Left Lower Outer Quadrant,[Not Applicable],[Not Applicable],[Not Available],[Not Applicable],0,[Not Available],[Not Available],[Not Available],[Not Applicable],Infiltrating Lobular Carcinoma,C50.9,8520/3,C50.9,YES,[Not Available],AAAU,[Not Available],[Not Applicable],[Not Available],3C,Breast
1,55262FCB-1B01-4480-B322-36570430C917,TCGA-3C-AALI,2014-7-28,NO,YES,-18538,FEMALE,Post (prior bilateral ovariectomy OR >12 mo si...,BLACK OR AFRICAN AMERICAN,NOT HISPANIC OR LATINO,No,No,TUMOR FREE,Alive,3801,[Not Applicable],YES,YES,[Not Applicable],2003,50,Core needle biopsy,[Not Applicable],Lumpectomy,[Not Available],Negative,[Not Available],[Not Applicable],[Not Available],Sentinel lymph node biopsy plus axillary disse...,[Not Available],[Not Available],YES,15,1,[Not Available],6th,T2,N1a,M0,...,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],Right Upper Outer Quadrant,[Not Applicable],[Not Applicable],[Not Available],[Not Applicable],0,[Not Available],[Not Available],[Not Available],[Not Applicable],Infiltrating Ductal Carcinoma,C50.9,8500/3,C50.9,YES,[Not Available],AALI,[Not Available],[Not Applicable],[Not Available],3C,Breast
2,427D0648-3F77-4FFC-B52C-8985

In [118]:
set(df["margin_status"].tolist())

{'Close', 'Negative', 'Positive', '[Not Available]', '[Unknown]'}

# Create LinkedData

In [116]:
kg = Graph()
kg.bind("sio", sio)
kg.bind("schema", schema)
kg.bind("obo", obo)
kg.bind("snomed", snomed)
for index, row in df.iterrows():
    # Patient URI
    patientURI = patientData[row["bcr_patient_uuid"]]
    kg.add((patientURI, RDF.type, schema.Patient)) 
    # gender
    if (row["gender"] == "FEMALE"):
        kg.add((patientURI, schema.gender, schema.Female))
        kg.add((patientURI, schema.gender, wikidata.Q6581072))
    elif(row["gender"] == "MALE"):
        kg.add((patientURI, schema.gender, schema.Female))
        kg.add((patientURI, schema.gender, wikidata.Q6581097))
        
    # diagnosis
    diagnosis = BNode()
    kg.add((patientURI, obo.RO_0016002, diagnosis))
    kg.add((diagnosis, RDF.type, obo.NCIT_C71732))
    kg.add((obo.NCIT_C71732, RDFS.label, Literal("triple-negative breast cancer", lang="en")))
    kg.add((obo.NCIT_C71732, SKOS.exactMatch, wikidata.Q7843332))
    
    # TCGA BAR code
    kg.add((diagnosis, DCTERMS.identifier, Literal(row["bcr_patient_barcode"],datatype=XSD.string)))
    kg.add((diagnosis, DC.identifier, tcga[row["bcr_patient_barcode"]]))
    
    # age at diagnosis
    age_at_diagnosis = BNode()
    kg.add((diagnosis, sio.SIO_000008 ,age_at_diagnosis ))
    #kg.parse(sio.SIO_000008)
    kg.add((age_at_diagnosis, RDF.value, Literal(row["age_at_diagnosis"], datatype=XSD.integer)))
    kg.add((age_at_diagnosis, RDF.type, obo.NCIT_C156420))
    kg.add((obo.NCIT_C156420, RDFS.label, Literal("Age at Diagnosis", lang="en")))
    # year of diagnosis
    year_of_diagnosis = BNode()
    kg.add((diagnosis, sio.SIO_000008, year_of_diagnosis))
    kg.add((year_of_diagnosis, RDF.value, Literal(row["initial_pathologic_dx_year"], datatype=XSD.gYear)))
    kg.add((year_of_diagnosis, RDF.type, obo.NCIT_C168823))
    kg.add((year_of_diagnosis, RDFS.label, Literal("Year of Diagnosis", lang="en")))
    
    # first surgical procedure
    first_surgical_procedure = BNode()
    if row["surgical_procedure_first"] == "Lumpectomy":
        kg.add((first_surgical_procedure, RDF.type, snomed["392021009"]))
        kg.add((first_surgical_procedure, obo.NCIT_R180, diagnosis))
    if row["surgical_procedure_first"] == "Modified Radical Mastectomy":
        kg.add((first_surgical_procedure, RDF.type, obo.NCIT_C15278))
        kg.add((first_surgical_procedure, obo.NCIT_R180, diagnosis))
    if row["surgical_procedure_first"] == "Modified Radical Mastectomy":
        kg.add((first_surgical_procedure, RDF.type, snomed["172043006"]))
        kg.add((first_surgical_procedure, obo.NCIT_R180, diagnosis))
        
    # method_initial_path_dx
    ## Using the same model as above
    ### {'Core needle biopsy',
    ### 'Cytology (e.g. Peritoneal or pleural fluid)',
    ###'Excisional Biopsy',
    ###'Fine needle aspiration biopsy',
    ###'Incisional Biopsy',
    ###'Other method, specify:',
    ###'Tumor resection',
    ###'[Discrepancy]',
    ### '[Not Available]'}
    if row["method_initial_path_dx"] == "Core needle biopsy":
        kg.add((first_surgical_procedure, RDF.type, snomed["9911007"])) ## Snomed has exact label
        kg.add((first_surgical_procedure, RDF.type, obo.NCIT_C15680)) ## NCIT lacks "Needle", but definition aligns
        kg.add((first_surgical_procedure, obo.NCIT_R180, diagnosis))
    if row["method_initial_path_dx"] == "Cytology":
        kg.add((first_surgical_procedure, RDF.type, obo.NCIT_C16491)) 
        kg.add((first_surgical_procedure, obo.NCIT_R180, diagnosis))
    if row["method_initial_path_dx"] == "Excisional Biopsy":
        kg.add((first_surgical_procedure, RDF.type, obo.NCIT_C15385)) 
        kg.add((first_surgical_procedure, obo.NCIT_R180, diagnosis))
    if row["method_initial_path_dx"] == "Fine needle biopsy":
        kg.add((first_surgical_procedure, RDF.type, snomed["48635004"])) ## Snomed has exact label
        kg.add((first_surgical_procedure, RDF.type, obo.NCIT_C15361)) ## Fine needle aspiration
        kg.add((first_surgical_procedure, obo.NCIT_R180, diagnosis))
    if row["method_initial_path_dx"] == "Incisional Biopsy":
        kg.add((first_surgical_procedure, RDF.type, obo.NCIT_C15386)) 
        kg.add((first_surgical_procedure, obo.NCIT_R180, diagnosis))
    if row["method_initial_path_dx"] == "Tumor resection":
        kg.add((first_surgical_procedure, RDF.type, obo.NCIT_C164212)) 
        kg.add((first_surgical_procedure, obo.NCIT_R180, diagnosis))
     
    # method_initial_path_dx_other
    ## Using the same model as above
    ###{'Biopsy not specified',
    ###'Biopsy, NOS',
    ###'Lumpectomy',
    ###'Modified Radical Masectomy',
    ###"Patey's Suregery",
    ###"Patey's Surgery",
    ###'SKIN BIOPSY',
    ###'Skin biopsy',
    ###'Ultrasound-guided biopsy',
    ###'Ultrasound-guided mammotome biopsy',
    ###'Wide local incision',
    ###'[Not Applicable]',
    ###'[Not Available]',
    ###'[Unknown]',
    ###'biopsy, NOS',
    ###'intraoperative examination',
    ###'stereotactic biopsy'}
    method_initial_path_dx_other = BNode()
    if row["method_initial_path_dx_other"] in  ["Biopsy not specified", 'Biopsy, NOS', 'biopsy, NOS'] :
        kg.add((method_initial_path_dx_other, RDF.type, obo.NCIT_C15189)) 
        kg.add((method_initial_path_dx_other, obo.NCIT_R180, diagnosis))
    if row["method_initial_path_dx_other"] == "Lumpectomy": 
        kg.add((method_initial_path_dx_other, RDF.type, obo.NCIT_C15755)) 
        kg.add((method_initial_path_dx_other, obo.NCIT_R180, diagnosis))
    if row["method_initial_path_dx_other"] == "Modified Radical Masectomy": 
        kg.add((method_initial_path_dx_other, RDF.type, obo.NCIT_C15278)) 
        kg.add((method_initial_path_dx_other, obo.NCIT_R180, diagnosis))
    if row["method_initial_path_dx_other"] in  ["Patey's Suregery", "Patey's Surgery"] :
        kg.add((method_initial_path_dx_other, RDF.type, snomed["395702000"])) 
        kg.add((method_initial_path_dx_other, obo.NCIT_R180, diagnosis))
    if row["method_initial_path_dx_other"] in  ["SKIN BIOPSY", "Skin biopsy"] :
        kg.add((method_initial_path_dx_other, RDF.type, obo.NCIT_C51692)) 
        kg.add((method_initial_path_dx_other, obo.NCIT_R180, diagnosis))
    if row["method_initial_path_dx_other"] in  ["Ultrasound-guided biopsy", "Ultrasound-guided mammotome biopsy"] :
        kg.add((method_initial_path_dx_other, RDF.type, obo.NCIT_C93022)) 
        kg.add((method_initial_path_dx_other, obo.NCIT_R180, diagnosis))
    if row["method_initial_path_dx_other"] == "Wide local incision": 
        kg.add((method_initial_path_dx_other, RDF.type, obo.NCIT_C94441)) ## NCIT does not contain WLincision, but WLexision
        kg.add((method_initial_path_dx_other, obo.NCIT_R180, diagnosis))
    if row["method_initial_path_dx_other"] == "intraoperative examination": 
        kg.add((method_initial_path_dx_other, RDF.type, snomed["73443006"])) ## snomed: Intraoperative state (finding)
        kg.add((method_initial_path_dx_other, obo.NCIT_R180, diagnosis))
    if row["method_initial_path_dx_other"] == "stereotactic biopsy": 
        kg.add((method_initial_path_dx_other, RDF.type, obo.NCIT_C51654)) ## snomed: Intraoperative state (finding)
        kg.add((method_initial_path_dx_other, obo.NCIT_R180, diagnosis)) 
    

In [117]:
print(kg.serialize())

@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix obo: <http://purl.obolibrary.org/obo/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema1: <http://schema.org/> .
@prefix sio: <http://semanticscience.org/resource/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix snomed: <http://snomed.info/id/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://leap.demonstrator.org/patient/001cef41-ff86-4d3f-a140-a647ac4b10a1> a schema1:Patient ;
    obo:RO_0016002 _:N167492f440e84bdf8d274e429d6667aa ;
    schema1:gender schema1:Female,
        <http://www.wikidata.org/entity/Q6581072> .

<http://leap.demonstrator.org/patient/0045349c-69d9-4306-a403-c9c1fa836644> a schema1:Patient ;
    obo:RO_0016002 _:N171e7da0dbfc43d19912e8bd8b380680 ;
    schema1:gender schema1:Female,
        <http://www.wikidata.org/entity/Q6581072> .

<http://leap

In [23]:
print(len(kg))

2
